In [ ]:
!pip install -q remotezip tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

## Packages

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections
import glob
from pathlib import Path
import os

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Preparing Data

In [ ]:
def get_class(fname):
    """ 
    Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Returns:
      Class that the file belongs to.
    """
    return fname.split('/')[4]

In [ ]:
def get_files_per_class(files):
    """ 
    Retrieve the files that belong to each class.

    Args:
      files: List of files in the dataset.

    Returns:
      Dictionary of class names (key) and files (values). 
    """
    files_for_class = collections.defaultdict(list)
    for fname in files:
        class_name = get_class(fname)
        files_for_class[class_name].append(fname)
    return files_for_class

In [ ]:
NUM_CLASSES = 10
FILES_PER_CLASS = 25

In [ ]:
files = glob.glob("drive/MyDrive/Data/*/*/*.mp4")
files.sort()
files[:10]

In [ ]:
files_for_class = get_files_per_class(files)
classes = list(files_for_class.keys())

In [ ]:
print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[0]]))

In [ ]:
def select_subset_of_classes(files_for_class, classes, files_per_class):
    """ 
    Create a dictionary with the class name and a subset of the files in that class.

    Args:
      files_for_class: Dictionary of class names (key) and files (values).
      classes: List of classes.
      files_per_class: Number of files per class of interest.

    Returns:
      Dictionary with class as key and list of specified number of video files in that class.
    """
    files_subset = dict()

    for class_name in classes:
        class_files = files_for_class[class_name]
        files_subset[class_name] = class_files[:files_per_class]

    return files_subset

In [ ]:
files_subset = select_subset_of_classes(files_for_class, classes[:NUM_CLASSES], FILES_PER_CLASS)
list(files_subset.keys())

## Create Frames from Each Video

In [ ]:
def format_frames(frame, output_size):
    """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

In [ ]:
def augment_frame(frame):
    """
    Performs data augmentation on a single frame.
    
    Args:
        frame : A single frame from the video.
        
    Return:
        The augmented frame.
    """
    
    # Randomly flip the frame horizontally
    if random.random() < 0.5:
        frame = cv2.flip(frame, 1)

    # Randomly rotate the frame
    angle = random.uniform(-10, 10)
    height, width = frame.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    frame = cv2.warpAffine(frame, rotation_matrix, (width, height))

    # Randomly crop the frame
    crop_size = int(min(frame.shape[:2]) * random.uniform(0.8, 1.0))
    x = random.randint(0, frame.shape[1] - crop_size)
    y = random.randint(0, frame.shape[0] - crop_size)
    frame = frame[y:y+crop_size, x:x+crop_size]

    return frame

In [ ]:
def frames_from_video_file(video_path, n_frames, output_size=(224, 224), frame_step=20, augment=False):
    """
    Creates frames from each video files present for each class.
    
    Args:
        video_path : File path to the video.
        n_frames : Number of frames to be created per video file.
        output_size : Pixel size of the output frame image.
        frame_step : The step size between consecutive frames.
        augment : Whether to perform data augmentation.
        
    Return:
        An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) + frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)

    # ret is a boolean indicating whether the read was successful, frame is the image itself.
    ret, frame = src.read()
    if augment:
        frame = augment_frame(frame)
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            if augment:
                frame = augment_frame(frame)
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))

    src.release()
    result = np.stack(result, axis=0)[..., [2, 1, 0]]

    return result

In [ ]:
video_path = "drive/MyDrive/Data/Test/LungePose/LungePose_02.mp4"

In [ ]:
sample_video = frames_from_video_file(video_path, n_frames = 16)
sample_video.shape

In [ ]:
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  return embed.embed_file('./animation.gif')

In [ ]:
to_gif(sample_video)